In [1]:
import json
import numpy as np
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)


In [2]:
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

In [13]:
data_df = pd.read_pickle('cleaned_list_df.pickle')
nosleep = data_df.T['nosleep'].tolist()
nosleep

worldnews = data_df.T['worldnews'].tolist()
worldnews

['so was there any posts left after the purge',
 'ok just for some context  facebook has an estimated  million fake user accounts every month about  billion messages are posted between users thats  billion per week billion  nobody gives a shit about the  million that some pr intern dug up when he reviewed some bots statistics as a great message to hype the tanking facebook popularity',
 'they still havent taken down my post about mars being as big as the moon s',
 'why does facebook have a fake news problem  because facebook is not a news site its social media its a place where anybody can express their opinion no matter how accurate or misled they are  you should not take anything posted on facebook as fact without verifying it though an actual news source  stop expecting a social media site to be a news policing site',
 'a lot of karens are going to be be saying they were hacked today',
 'i bought a mask that arrived in a box that had a warning on it saying it wasnt for avoiding coro

In [4]:
politics = data_df.T['politics'].tolist()

In [6]:
vectorizer = CountVectorizer(stop_words='english',
                             binary=True, ngram_range=(1,2))

doc_word = vectorizer.fit_transform(nosleep)
words = list(np.asarray(vectorizer.get_feature_names())) 
doc_word

<101141x68226 sparse matrix of type '<class 'numpy.int64'>'
	with 140586 stored elements in Compressed Sparse Row format>

In [12]:
cvn = TfidfVectorizer(ngram_range=(1,1))
data_cvn = cvn.fit_transform(nosleep)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
# data_dtmn.columns = data_df.index
data_dtmn

,0,1,2,3,4,5,6,7,8,9,...,101131,101132,101133,101134,101135,101136,101137,101138,101139,101140
aa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaaaaaaaaarrrrggg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaagh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zune,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ántiįhnii,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ಠಠ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(doc_word))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in vectorizer.vocabulary_.items())
word2id = dict((k, v) for k, v in vectorizer.vocabulary_.items())

In [9]:
# Let's start with 2 topics
ldan = models.LdaMulticore(corpus=corpusn, num_topics=10, id2word=id2wordn, passes=50)
ldan.print_topics()

INFO - 21:23:15: using symmetric alpha at 0.1
INFO - 21:23:15: using symmetric eta at 0.1
INFO - 21:23:15: using serial LDA version on this node
INFO - 21:23:15: running online LDA training, 10 topics, 50 passes over the supplied corpus of 68226 documents, updating every 30000 documents, evaluating every ~68226 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 21:23:15: training LDA model using 15 processes
INFO - 21:23:16: PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/68226, outstanding queue size 1
INFO - 21:23:16: PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/68226, outstanding queue size 2
INFO - 21:23:16: PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/68226, outstanding queue size 3
INFO - 21:23:16: PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/68226, outstanding queue size 4
INFO - 21:23:16: PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/68226, outstanding queue size 5
INFO - 21:

INFO - 21:23:17: topic #2 (0.100): 0.013*"big hairy" + 0.008*"called jasonthats" + 0.007*"camera beeping" + 0.007*"assimilating victims" + 0.005*"ana" + 0.004*"bc" + 0.003*"allows conditions" + 0.003*"athame" + 0.003*"cage bit" + 0.003*"bbestie"
INFO - 21:23:17: topic #0 (0.100): 0.020*"admirer" + 0.004*"acceptable" + 0.004*"called calm" + 0.004*"better solution" + 0.004*"attention assert" + 0.004*"accident ordered" + 0.004*"awe" + 0.003*"best irish" + 0.003*"bruh working" + 0.003*"avoid used"
INFO - 21:23:17: topic diff=0.030529, rho=0.176465
INFO - 21:23:17: -10.323 per-word bound, 1281.1 perplexity estimate based on a held-out corpus of 226 documents with 395 words
INFO - 21:23:17: PROGRESS: pass 1, dispatched chunk #0 = documents up to #2000/68226, outstanding queue size 1
INFO - 21:23:17: PROGRESS: pass 1, dispatched chunk #1 = documents up to #4000/68226, outstanding queue size 2
INFO - 21:23:17: PROGRESS: pass 1, dispatched chunk #2 = documents up to #6000/68226, outstanding que

INFO - 21:23:19: topic #7 (0.100): 0.006*"anybody love" + 0.005*"allusion class" + 0.005*"called edit" + 0.004*"best interests" + 0.004*"assuming read" + 0.004*"allowed speak" + 0.003*"bc dont" + 0.003*"agree readym" + 0.003*"baby gave" + 0.003*"called calm"
INFO - 21:23:19: topic #8 (0.100): 0.017*"awwloved want" + 0.012*"ask shared" + 0.009*"allowed switch" + 0.008*"camera fell" + 0.005*"award support" + 0.004*"bois knew" + 0.004*"answer just" + 0.004*"bring experience" + 0.004*"aww poor" + 0.004*"biggest fears"
INFO - 21:23:19: topic #4 (0.100): 0.007*"called cops" + 0.005*"alexei fanatic" + 0.004*"campsite ill" + 0.004*"archangelsthe" + 0.004*"assure pagan" + 0.004*"antimasker" + 0.004*"asss happened" + 0.003*"balance think" + 0.003*"cancelled" + 0.003*"antique dolls"
INFO - 21:23:19: topic diff=0.029874, rho=0.166406
INFO - 21:23:19: -9.970 per-word bound, 1003.3 perplexity estimate based on a held-out corpus of 226 documents with 395 words
INFO - 21:23:19: PROGRESS: pass 2, dispa

INFO - 21:23:21: topic #0 (0.100): 0.027*"admirer" + 0.007*"better solution" + 0.006*"called calm" + 0.006*"attention assert" + 0.006*"accident ordered" + 0.005*"acceptable" + 0.005*"best irish" + 0.005*"awe" + 0.005*"bruh working" + 0.004*"allthetime thing"
INFO - 21:23:21: topic #6 (0.100): 0.010*"avoid used" + 0.010*"brian lecture" + 0.006*"bullies harassed" + 0.006*"bit ruthlessness" + 0.005*"appearance kates" + 0.005*"admirer" + 0.005*"antimateriel" + 0.004*"aye" + 0.004*"big hairy" + 0.004*"adventures living"
INFO - 21:23:21: topic #8 (0.100): 0.019*"awwloved want" + 0.014*"ask shared" + 0.009*"allowed switch" + 0.009*"camera fell" + 0.005*"award support" + 0.005*"bois knew" + 0.005*"answer just" + 0.005*"bring experience" + 0.004*"biggest fears" + 0.004*"asks"
INFO - 21:23:21: topic #7 (0.100): 0.006*"anybody love" + 0.006*"allusion class" + 0.006*"best interests" + 0.005*"called edit" + 0.004*"allowed speak" + 0.004*"bc dont" + 0.004*"assuming read" + 0.004*"agree readym" + 0.0

INFO - 21:23:23: topic #7 (0.100): 0.007*"best interests" + 0.006*"anybody love" + 0.006*"allusion class" + 0.005*"called edit" + 0.004*"allowed speak" + 0.004*"assuming read" + 0.004*"bc dont" + 0.004*"agree readym" + 0.003*"biker thought" + 0.003*"baby gave"
INFO - 21:23:23: topic #2 (0.100): 0.025*"big hairy" + 0.015*"assimilating victims" + 0.011*"camera beeping" + 0.010*"called jasonthats" + 0.009*"bc" + 0.007*"ana" + 0.006*"cage bit" + 0.005*"athame" + 0.005*"called" + 0.004*"bbestie"
INFO - 21:23:23: topic #0 (0.100): 0.029*"admirer" + 0.008*"better solution" + 0.007*"called calm" + 0.007*"attention assert" + 0.006*"accident ordered" + 0.006*"best irish" + 0.006*"acceptable" + 0.005*"bruh working" + 0.005*"awe" + 0.004*"allthetime thing"
INFO - 21:23:23: topic #8 (0.100): 0.021*"awwloved want" + 0.016*"ask shared" + 0.010*"allowed switch" + 0.009*"camera fell" + 0.006*"award support" + 0.005*"bois knew" + 0.005*"answer just" + 0.005*"bring experience" + 0.005*"biggest fears" + 0

...

INFO - 21:24:29: topic #5 (0.100): 0.008*"caine keeping" + 0.007*"betting killed" + 0.007*"breath thought" + 0.006*"afraid nature" + 0.005*"best vague" + 0.005*"adopted kidyeah" + 0.005*"beepbeepbeepbeepbeepbeep smell" + 0.005*"act appear" + 0.005*"arcana guess" + 0.005*"bryans companions"
INFO - 21:24:29: topic #2 (0.100): 0.033*"big hairy" + 0.020*"assimilating victims" + 0.014*"camera beeping" + 0.011*"called jasonthats" + 0.011*"bc" + 0.007*"ana" + 0.007*"cage bit" + 0.007*"called" + 0.006*"athame" + 0.006*"advanced universe"
INFO - 21:24:29: topic diff=0.003407, rho=0.111034
INFO - 21:24:30: -9.334 per-word bound, 645.3 perplexity estimate based on a held-out corpus of 226 documents with 395 words
INFO - 21:24:30: merging changes from 6226 documents into a model of 68226 documents
INFO - 21:24:30: topic #4 (0.100): 0.014*"called cops" + 0.007*"alexei fanatic" + 0.007*"campsite ill" + 0.006*"assure pagan" + 0.006*"archangelsthe" + 0.005*"cancelled" + 0.005*"asked time" + 0.005*"bal

INFO - 21:24:31: topic #9 (0.100): 0.010*"abroad" + 0.010*"asks away" + 0.009*"barter" + 0.006*"blanks post" + 0.006*"address challenging" + 0.005*"aliens steal" + 0.005*"anagram come" + 0.005*"ahhhh daughter" + 0.005*"astute" + 0.005*"aspects home"
INFO - 21:24:31: topic #3 (0.100): 0.011*"able youd" + 0.009*"atinlay" + 0.007*"air" + 0.007*"ass going" + 0.007*"awkward encounter" + 0.006*"bystander target" + 0.006*"boundaries weak" + 0.006*"allows separate" + 0.005*"ancient spirit" + 0.005*"anymore guess"
INFO - 21:24:31: topic #5 (0.100): 0.009*"caine keeping" + 0.008*"breath thought" + 0.007*"betting killed" + 0.006*"afraid nature" + 0.005*"best vague" + 0.005*"adopted kidyeah" + 0.005*"beepbeepbeepbeepbeepbeep smell" + 0.005*"aspects thing" + 0.005*"arcana guess" + 0.005*"act appear"
INFO - 21:24:31: topic diff=0.003321, rho=0.110356
INFO - 21:24:31: -9.308 per-word bound, 633.7 perplexity estimate based on a held-out corpus of 226 documents with 395 words
INFO - 21:24:31: merging c

INFO - 21:24:33: topic #2 (0.100): 0.033*"big hairy" + 0.020*"assimilating victims" + 0.014*"camera beeping" + 0.011*"called jasonthats" + 0.011*"bc" + 0.007*"cage bit" + 0.007*"ana" + 0.007*"called" + 0.006*"athame" + 0.006*"amazingly tricky"
INFO - 21:24:33: topic #4 (0.100): 0.014*"called cops" + 0.007*"alexei fanatic" + 0.007*"campsite ill" + 0.006*"archangelsthe" + 0.006*"assure pagan" + 0.006*"cancelled" + 0.005*"backup" + 0.005*"asked time" + 0.005*"argue" + 0.005*"balance think"
INFO - 21:24:33: topic #7 (0.100): 0.010*"best interests" + 0.007*"anybody love" + 0.007*"called edit" + 0.007*"allowed speak" + 0.006*"allusion class" + 0.005*"assuming read" + 0.005*"barely" + 0.005*"baby gave" + 0.005*"bc dont" + 0.004*"biker thought"
INFO - 21:24:33: topic #8 (0.100): 0.026*"awwloved want" + 0.021*"ask shared" + 0.011*"allowed switch" + 0.011*"camera fell" + 0.008*"bois knew" + 0.007*"award support" + 0.007*"answer just" + 0.006*"biggest fears" + 0.006*"bring experience" + 0.006*"aw

INFO - 21:24:34: merging changes from 30000 documents into a model of 68226 documents
INFO - 21:24:34: topic #3 (0.100): 0.012*"able youd" + 0.009*"atinlay" + 0.007*"air" + 0.007*"ass going" + 0.007*"awkward encounter" + 0.006*"bystander target" + 0.006*"boundaries weak" + 0.006*"allows separate" + 0.005*"ancient spirit" + 0.005*"anymore guess"
INFO - 21:24:34: topic #4 (0.100): 0.014*"called cops" + 0.007*"alexei fanatic" + 0.006*"campsite ill" + 0.006*"archangelsthe" + 0.006*"assure pagan" + 0.006*"cancelled" + 0.005*"asked time" + 0.005*"backup" + 0.005*"argue" + 0.005*"biting throat"
INFO - 21:24:34: topic #0 (0.100): 0.036*"admirer" + 0.011*"called calm" + 0.010*"better solution" + 0.010*"attention assert" + 0.008*"bruh working" + 0.007*"best irish" + 0.007*"accident ordered" + 0.007*"acceptable" + 0.006*"allthetime thing" + 0.006*"away place"
INFO - 21:24:34: topic #7 (0.100): 0.010*"best interests" + 0.007*"anybody love" + 0.007*"called edit" + 0.007*"allowed speak" + 0.006*"all

[(0,
  '0.035*"admirer" + 0.011*"called calm" + 0.010*"better solution" + 0.010*"attention assert" + 0.008*"bruh working" + 0.007*"best irish" + 0.007*"accident ordered" + 0.006*"acceptable" + 0.006*"away place" + 0.006*"allthetime thing"'),
 (1,
  '0.007*"brass knuckles" + 0.007*"bullet attached" + 0.007*"arent rewards" + 0.007*"ass killed" + 0.007*"ally im" + 0.006*"appropriately responding" + 0.006*"architect" + 0.006*"believe instance" + 0.006*"assembly" + 0.005*"asses mess"'),
 (2,
  '0.033*"big hairy" + 0.020*"assimilating victims" + 0.014*"camera beeping" + 0.011*"bc" + 0.011*"called jasonthats" + 0.008*"cage bit" + 0.007*"ana" + 0.007*"called" + 0.006*"athame" + 0.006*"amazingly tricky"'),
 (3,
  '0.011*"able youd" + 0.009*"atinlay" + 0.007*"air" + 0.007*"ass going" + 0.007*"awkward encounter" + 0.006*"bystander target" + 0.006*"allows separate" + 0.006*"boundaries weak" + 0.005*"ancient spirit" + 0.005*"anymore guess"'),
 (4,
  '0.014*"called cops" + 0.007*"alexei fanatic" + 0

In [11]:
# doct_topic = lsa_model.fit_transform(doc_word)

NameError: name 'lsa_model' is not defined

In [24]:
# ldan.save('nosleep_idf.model')

INFO - 14:17:02: saving LdaState object under nosleep_idf.model.state, separately None
INFO - 14:17:02: saved nosleep_idf.model.state
INFO - 14:17:02: saving LdaMulticore object under nosleep_idf.model, separately ['expElogbeta', 'sstats']
INFO - 14:17:02: storing np array 'expElogbeta' to nosleep_idf.model.expElogbeta.npy
INFO - 14:17:02: not storing attribute state
INFO - 14:17:02: not storing attribute dispatcher
INFO - 14:17:02: not storing attribute id2word
INFO - 14:17:02: saved nosleep_idf.model


In [25]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [26]:
from gensim import corpora
d = corpora.Dictionary()
d.id2token = id2wordn
d.token2id = word2id

## 10 topics, TF-IDF'ed

In [27]:
vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

## 10 topics, CountVectorized

In [20]:
vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

INFO - 14:13:40: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 14:13:40: NumExpr defaulting to 8 threads.
